# Exploratory Data Analysis (EDA)
This notebook combines all features (old and new), explores them, visualizes distributions and correlations, and keeps only relevant features based on correlation with the target.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from textblob import TextBlob
import warnings
from datetime import datetime, timedelta
from utils import *
warnings.filterwarnings('ignore')

## 1. Load Combined Features

In [ ]:
TICKERS = {
    # === CORE OIL & ENERGY ===
    'WTI_Crude': 'CL=F',           # Target variable
    'Brent_Oil': 'BZ=F',           # Alternative oil benchmark
    'Natural_Gas': 'NG=F',         # Natural gas futures
    'RBOB_Gasoline': 'RB=F',       # Gasoline futures
    'Heating_Oil': 'HO=F',         # Heating oil futures

    # === EQUITY INDICES ===
    'SP500': '^GSPC',              # S&P 500
    'NASDAQ': '^IXIC',             # NASDAQ
    'Russell2000': '^RUT',         # Small cap
    'FTSE100': '^FTSE',            # UK market
    'DAX': '^GDAXI',               # German market
    'Nikkei': '^N225',             # Japanese market

    # === CURRENCIES ===
    'DXY': 'DX-Y.NYB',             # US Dollar Index
    'EUR_USD': 'EURUSD=X',         # Euro/USD
    'GBP_USD': 'GBPUSD=X',         # British Pound/USD
    'CAD_USD': 'CADUSD=X',         # Canadian Dollar/USD (oil exporter)
    'NOK_USD': 'NOKUSD=X',         # Norwegian Krone/USD (oil exporter)

    # === INTEREST RATES & BONDS ===
    'US2Y': '^IRX',                # 2-year Treasury
    'US5Y': '^FVX',                # 5-year Treasury
    'US10Y': '^TNX',               # 10-year Treasury
    'US30Y': '^TYX',               # 30-year Treasury

    # === COMMODITIES ===
    'Gold': 'GC=F',                # Gold futures
    'Silver': 'SI=F',              # Silver futures
    'Copper': 'HG=F',              # Copper futures

    # === VOLATILITY & RISK ===
    'VIX': '^VIX',                 # Volatility Index

    # === SECTOR ETFS ===
    'Energy_ETF': 'XLE',           # Energy sector ETF
    'Oil_ETF': 'USO',              # Oil ETF
    'Transportation': 'IYT',        # Transportation ETF
    'Materials': 'XLB',            # Materials sector
    'Emerging_Markets': 'EEM'       # Emerging markets
}

# Date configuration
START_DATE = '2000-01-01'
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')

print(f"📊 Configured {len(TICKERS)} market indicators")
print(f"📅 Date range: {START_DATE} to {END_DATE}")

## 2. Basic Data Overview

In [ ]:
market_data, failed_downloads = download_market_data(TICKERS, START_DATE, END_DATE)

if market_data is None:
  print("❌ Failed to download any market data. Exiting.")

In [ ]:
market_data.head()

## 3. Missing Value Analysis

In [ ]:
processed_data = clean_and_process_data(market_data, add_sentiment=True)
processed_data.head()

## 4. Feature Distributions

## 5. Correlation Heatmap

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

def analyze_feature_relationships_with_oil(data, target_col='WTI_Crude', threshold=0.3, top_n=10):
    """
    Perform feature correlation and importance analysis with respect to oil prices.

    Parameters:
    -----------
    data : pd.DataFrame
        Market data (features as columns).
    target_col : str
        Name of the target feature (default: 'WTI_Crude').
    threshold : float
        Absolute correlation threshold to select relevant features.
    top_n : int
        Number of top correlated features to display/analyze.

    Returns:
    --------
    selected_features : list
        List of selected feature names (including target).
    correlations : pd.Series
        Series of feature correlations with target_col.
    """
    if target_col not in data.columns:
        print(f"❌ Target column '{target_col}' not found in data.")
        return None, None

    print(f"\n🎯 Analyzing feature relationships with {target_col}...")

    # 1. Correlation Matrix
    corr_matrix = data.corr()
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr_matrix, cmap='coolwarm', center=0, annot=False)
    plt.title("📌 Feature Correlation Matrix")
    plt.tight_layout()
    plt.show()

    # 2. Correlation with Target
    correlations = corr_matrix[target_col].drop(target_col)
    correlations_abs = correlations.abs().sort_values(ascending=False)

    print(f"\n🔥 Top {top_n} Correlations with {target_col}:")
    for i, (feature, abs_corr) in enumerate(correlations_abs.head(top_n).items(), 1):
        actual_corr = correlations[feature]
        print(f"   {i:2}. {feature:25}: {actual_corr:+.3f} (|{abs_corr:.3f}|)")

    # 3. Bar Chart of Top-N Correlations
    plt.figure(figsize=(10, 6))
    top_corr = correlations_abs.head(top_n)
    sns.barplot(x=top_corr.values, y=top_corr.index, orient='h', palette='coolwarm')
    plt.title(f"📊 Top {top_n} Absolute Correlations with {target_col}")
    plt.xlabel("Correlation (absolute)")
    plt.tight_layout()
    plt.show()

    # 4. Feature Selection
    selected_features_corr = correlations_abs[correlations_abs >= threshold]
    selected_features = [target_col] + list(selected_features_corr.index)

    print(f"\n✅ Selected {len(selected_features)-1} features with |correlation| ≥ {threshold}")

    # 5. Pairplot for selected features
    if len(selected_features) <= 10:
        sns.pairplot(data[selected_features].dropna())
        plt.suptitle("📷 Pairwise Relationships (Selected Features)", y=1.02)
        plt.show()
    else:
        print("📷 Skipping pairplot (too many features selected).")

    # 6. Feature Importance (Random Forest)
    print("\n🌲 Calculating feature importances (Random Forest)...")
    feature_data = data[selected_features].dropna()
    X = feature_data.drop(columns=target_col)
    y = feature_data[target_col]

    # Normalize features for importance comparison
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_scaled, y)

    importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=True)

    plt.figure(figsize=(10, 6))
    importances.plot(kind='barh', color='teal')
    plt.title("🌟 Feature Importances (Random Forest)")
    plt.xlabel("Importance Score")
    plt.tight_layout()
    plt.show()

    # 7. Histogram/Distribution of Selected Features
    print("\n📈 Plotting distributions of selected features...")
    selected_data = data[selected_features].dropna()
    selected_data.hist(figsize=(15, 10), bins=30, layout=(len(selected_features) // 3 + 1, 3))
    plt.suptitle("📊 Distributions of Selected Features", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
        # 8. Scatterplots vs. Target
    print("\n📌 Scatterplots of selected features vs. target...")
    max_plots = min(9, len(selected_features) - 1)
    fig, axes = plt.subplots(nrows=(max_plots + 2) // 3, ncols=3, figsize=(15, 10))
    axes = axes.flatten()

    for i, feature in enumerate(selected_features_corr.head(max_plots).index):
        ax = axes[i]
        sns.scatterplot(data=data, x=feature, y=target_col, ax=ax, alpha=0.5)
        corr_val = correlations[feature]
        ax.set_title(f"{feature} (r = {corr_val:.2f})")

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f"📉 Scatterplots vs. {target_col}", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()


    return selected_features, correlations


In [ ]:
selected_features, correlations = analyze_feature_relationships_with_oil(
            processed_data, target_col='WTI_Crude', threshold=0.3
        )
final_data = processed_data[selected_features].copy()

## 6. Select Highly Correlated Features

In [ ]:
print(f"\n" + "=" * 60)
print(f"📋 FINAL DATASET SUMMARY")
print(f"=" * 60)
print(f"   • Shape: {final_data.shape[0]:,} rows × {final_data.shape[1]:,} columns")
print(f"   • Date range: {final_data.index.min().strftime('%Y-%m-%d')} to {final_data.index.max().strftime('%Y-%m-%d')}")
print(f"   • Features included: {final_data.shape[1]:,}")

## 7. Save Relevant Features

In [ ]:
# Save the filtered relevant features
final_data.to_csv("relevant_features.csv")
print("Saved to relevant_features.csv")

In [ ]:
final_data.describe()